In [1]:
!pip install protobuf==3.20.3 trl peft accelerate bitsandbytes unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
   

In [1]:
from unsloth import FastLanguageModel
import json
import torch
from datasets import Dataset
from transformers import TrainingArguments, AutoTokenizer
from trl import SFTTrainer

def interactive_test(model, tokenizer):
    """
    Creates an interactive loop to test the model with user prompts.
    """
    print("\n--- Interactive Model Test ---")
    print("Enter a prompt to test the fine-tuned model.")
    print("Type 'save' to finish testing and save the GGUF model.")
    print("Type 'cancel' to exit without saving.")
    print("------------------------------------")

    while True:
        # Get input from the user
        user_input = input("\nPrompt: ")

        # Check for control commands
        if user_input.lower() == "save":
            print("\nProceeding to save the model...")
            return True  # Signal to continue and save
        elif user_input.lower() == "cancel":
            print("\nCanceling save. Exiting script.")
            return False  # Signal to exit without saving

        # Prepare the input for the model
        messages = [
            {"role": "user", "content": user_input},
        ]
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        # Generate a response
        outputs = model.generate(input_ids=inputs, max_new_tokens=256, use_cache=True)
        response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Clean up the output to only show the assistant's part
        # Note: The split logic may vary slightly based on the exact model output format.
        # This is a common way to parse it.
        assistant_response = response_text.split("<|assistant|>")
        if len(assistant_response) > 1:
            clean_response = assistant_response[1].strip()
        else:
            # Fallback for models that don't add the user prompt to the output
            # or have a different format.
            if "user" in response_text and "assistant" in response_text:
                 clean_response = response_text.split("assistant")[-1].strip()
            else:
                 clean_response = response_text

        print(f"Model: {clean_response}")

# Import Dataset
try:
    with open("data.json", "r") as f:
        file = json.load(f)
    print("Successfully loaded data.json.")
    print("Sample record:", file[1])
except FileNotFoundError:
    print("Error: data.json not found. Please upload it to the Colab session.")
except Exception as e:
    print(f"An error occurred: {e}")


# Load the base model
model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

# Create the dataset directly from the loaded file
dataset = Dataset.from_list(file)

# Formatting prompts so they can be sent like this {"context": "今天天气不错，不冷不热。晚饭后，我们决定去公园散步。那里的风景很优美，空气也很新鲜。", "target_sentence": "真是个锻练身体的好地方。"}
def format_prompts(batch):
    """
    Takes a batch of examples and returns a list of formatted strings
    in a chat format where the user input is a JSON object.
    """
    contexts = batch["context"]
    target_sentences = batch["target_sentence"]
    outputs = batch["output"]

    texts = []
    for context, target, output in zip(contexts, target_sentences, outputs):
        input_json = {
            "context": context,
            "target_sentence": target
        }

        # The user provides the simple JSON, and the assistant provides the correction JSON followed by the EOS token.
        prompt = f"<|user|>\n{json.dumps(input_json, ensure_ascii=False)}\n<|assistant|>\n{json.dumps(output, ensure_ascii=False)}<|endoftext|>"
        texts.append(prompt)

    return texts

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Training arguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    formatting_func=format_prompts,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False, # Important for Colab
        report_to="none",
    ),
)

# Train the model
print("\n--- Starting Model Training ---")
trainer_stats = trainer.train()
print("--- Model Training Finished ---")


# Merge and save the 16-bit model
merged_model_path = "merged_16bit_model"
model.save_pretrained_merged(merged_model_path, tokenizer, save_method="merged_16bit")

# Reload the merged model for testing
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=merged_model_path,
    dtype=dtype,
    load_in_4bit=False,
)

# Run interactive test
should_save = interactive_test(model, tokenizer)

# Save the final GGUF model if requested
if should_save:
    model.save_pretrained_gguf(
        "gguf_model", tokenizer, quantization_method="q4_k_m"
    )
    print("\nGGUF model saved successfully in the 'gguf_model' file.")
    print("You can download it from the file browser on the left.")
else:
    print("\nExiting. The final GGUF model was not saved.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Successfully loaded data.json.
Sample record: {'context': '这个星期工作太忙了，我感觉很累。终于等到周末可以放松一下了。明天是周末，你有什么计划吗？', 'target_sentence': '我打算和朋友一起去看电影。', 'output': {'is_correction_needed': False, 'corrected_sentence': '我打算和朋友一起去看电影。', 'reasoning': 'The sentence is grammatically correct and fits the context. No changes are needed.'}}
==((====))==  Unsloth 2025.10.1: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

 Done.
Unsloth: Saving gguf_model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving gguf_model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at gguf_model into f16 GGUF format.
The output location will be /content/gguf_model/unsloth.F16.gguf
This might take 3 minutes...


Unsloth: Extending gguf_model/tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


INFO:hf-to-gguf:Loading model: gguf_model
INFO:hf-to-gguf:Model architecture: MistralForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_v.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf: